In [8]:
from flask import Flask,jsonify,render_template
from flask_cors import CORS
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [18]:
with open("../restaurant_data.json", 'r') as file:
    data_but_like_the_json_one = json.load(file)

In [19]:
def queried_to_obj(sql_version):
    return {property.name: getattr(sql_version, property.name) for property in sql_version.__table__.columns}

def pull_vote_totals(types):
    session = Session()
    boop = session.query(types).all()
    democracy = {"mexican":0, "italian": 0, "thai":0}
    for i in boop:        
        democracy[i.restaurant_type] = i.votes_cast   
    session.close()
    return democracy
    

def add_vote_to_db(vote,types):
    session = Session()
    tp = session.query(types).filter(types.restaurant_type == vote).first()
    tp.votes_cast += 1
    session.commit()
    session.close()
    return json.dumps("vote added to db")

In [20]:
engine_main = create_engine("sqlite:///Resources/main_data.sqlite")

base_main = automap_base()
base_main.prepare(autoload_with=engine_main)

eatery = base_main.classes.restaurant_data

Session_main = sessionmaker(bind=engine_main)
session_main = Session_main()

data_but_from_sql_pre = session_main.query(eatery).all()

the_places = list(map(queried_to_obj, data_but_from_sql_pre))
data_but_from_sql = json.dumps(the_places)
session_main.close()

In [21]:
engine = create_engine("sqlite:///Resources/votes.sqlite")

base = automap_base()
base.prepare(autoload_with=engine)

cuisines = base.classes.Favorite_Cuisines

Session = sessionmaker(bind=engine) 

In [46]:
app = Flask(__name__)
CORS(app,resources={r"/*": {"origins": ["http://127.0.0.1:5000"]}})


@app.route("/")
def home():
    return render_template('index.html')

@app.route("/map")
def dontwait():
    return render_template('map.html')

@app.route("/chart")
def justkiddingyoucanwait():
    return render_template('chart.html')

@app.route("/btc")
def nvm():
    return render_template('btc.html')

@app.route("/vote<vote>")
def noway(vote):
    add_vote_to_db(vote,cuisines)
    return vote
    
@app.route("/voteTot")
def jose():
    voteTots = pull_vote_totals(cuisines)
    return json.dumps(voteTots)

@app.route("/data")
def josephine():
    return json.dumps(data_but_from_sql)


if(__name__=="__main__"):
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2023 19:25:21] "GET /chart HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 19:25:28] "GET /data HTTP/1.1" 200 -
